# Querying of BioPAX exports of pathway databases: Get the number of instances of main BioPAX classes

#### Import libraries

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import IPython
import subprocess
import time

#### Define the URL of local SPARQL endpoints

In [2]:
endpointURL_reactome_v87 = "http://localhost:3030/reactome_v87/query"
endpointURL_reactome_pc = "http://localhost:3030/reactome_pc/query"
endpointURL_panther_pc = "http://localhost:3030/panther_pc/query"
endpointURL_pathbank_pc = "http://localhost:3030/pathbank_pc/query"
endpointURL_humancyc_pc = "http://localhost:3030/humancyc_pc/query"
endpointURL_kegg_pc = "http://localhost:3030/kegg_pc/query"
endpointURL_pid_pc = "http://localhost:3030/pid_pc/query"
endpointURL_inoh_pc = "http://localhost:3030/inoh_pc/query"
endpointURL_netpath_pc = "http://localhost:3030/netpath_pc/query"
rdfFormat = "turtle"

#### Define RDF prefixes for SPARQL queries

In [3]:
reactomeVersion = 87
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

#### Function to display the results of SPARQL queries

In [4]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

## Count number of instances of BioPAX classes in BioPAX exports of pathway databases

TODO: change the path to fuseki server and BioPAX files + BioPAX ontology in the command line to launch the SPARQL endpoint

#### Define SPARQL query

In [5]:
query = """
SELECT ?class_of_interest (COUNT(DISTINCT ?instances) AS ?nb_instances) 
WHERE {  
  # BioPAX classe for which we want to count the number of instances
  VALUES ?class_of_interest { bp3:SmallMolecule bp3:Interaction bp3:Pathway bp3:PathwayStep bp3:BiochemicalReaction bp3:Protein bp3:Dna bp3:Rna }
  OPTIONAL {
  ?instances rdf:type/(rdfs:subClassOf*) ?class_of_interest .
  }
}
GROUP BY ?class_of_interest
"""

#### Query on Reactome BioPAX standalone export version 87

In [6]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v87/Homo_sapiens.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_v87']
process = subprocess.Popen(command)
time.sleep(60)
# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_v87)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_v87_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:25:11 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v87/Homo_sapiens.owl
18:25:13 WARN  riot            :: [line: 67280, col: 52] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:25:32 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:25:32 INFO  Server          :: Running in read-only mode for /reactome_v87
18:25:33 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:25:33 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:25:33 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:25:33 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:25:33 INFO  Server       

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,14709
http://www.biopax.org/release/biopax-level3.owl#Pathway,2673
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,17383
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,5196
http://www.biopax.org/release/biopax-level3.owl#Rna,654
http://www.biopax.org/release/biopax-level3.owl#Interaction,23577
http://www.biopax.org/release/biopax-level3.owl#Protein,31649
http://www.biopax.org/release/biopax-level3.owl#Dna,1479


18:26:21 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome_v87/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

#### Query on BioPAX export of Reactome from PathwayCommons

In [7]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.reactome.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:27:30 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.reactome.BIOPAX.owl
18:27:32 WARN  riot            :: [line: 100212, col: 81] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:27:51 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:27:51 INFO  Server          :: Running in read-only mode for /reactome_pc
18:27:51 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:27:51 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:27:51 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:27:51 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:27:51 INFO  Server    

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,12184
http://www.biopax.org/release/biopax-level3.owl#Pathway,2272
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,14460
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,3983
http://www.biopax.org/release/biopax-level3.owl#Rna,312
http://www.biopax.org/release/biopax-level3.owl#Interaction,19640
http://www.biopax.org/release/biopax-level3.owl#Protein,26582
http://www.biopax.org/release/biopax-level3.owl#Dna,890


18:28:39 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of Panther from PathwayCommons

In [8]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.panther.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/panther_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_panther_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/panther_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:29:47 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.panther.BIOPAX.owl
18:29:48 WARN  riot            :: [line: 80968, col: 73] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:29:50 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:29:50 INFO  Server          :: Running in read-only mode for /panther_pc
18:29:50 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:29:50 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:29:50 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:29:50 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:29:51 INFO  Server       

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,2222
http://www.biopax.org/release/biopax-level3.owl#Pathway,295
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,0
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,1936
http://www.biopax.org/release/biopax-level3.owl#Rna,0
http://www.biopax.org/release/biopax-level3.owl#Interaction,5982
http://www.biopax.org/release/biopax-level3.owl#Protein,5342
http://www.biopax.org/release/biopax-level3.owl#Dna,0


18:30:48 INFO  Fuseki          :: [2] GET http://localhost:3030/panther_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.react

#### Query on BioPAX export of PathBank from PathwayCommons

In [9]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pathbank.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pathbank_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pathbank_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pathbank_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:31:50 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pathbank.BIOPAX.owl
18:31:51 WARN  riot            :: [line: 89570, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:31:53 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:31:53 INFO  Server          :: Running in read-only mode for /pathbank_pc
18:31:53 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:31:53 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:31:53 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:31:53 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:31:54 INFO  Server     

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,2022
http://www.biopax.org/release/biopax-level3.owl#Pathway,1295
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,0
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,3326
http://www.biopax.org/release/biopax-level3.owl#Rna,52
http://www.biopax.org/release/biopax-level3.owl#Interaction,5291
http://www.biopax.org/release/biopax-level3.owl#Protein,2471
http://www.biopax.org/release/biopax-level3.owl#Dna,9


18:32:51 INFO  Fuseki          :: [2] GET http://localhost:3030/pathbank_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of HumanCyc from PathwayCommons

In [10]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.humancyc.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/humancyc_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_humancyc_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/humancyc_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:33:53 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.humancyc.BIOPAX.owl
18:33:54 WARN  riot            :: [line: 108021, col: 73] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:33:57 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:33:57 INFO  Server          :: Running in read-only mode for /humancyc_pc
18:33:57 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:33:57 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:33:57 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:33:57 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:33:57 INFO  Server    

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,1805
http://www.biopax.org/release/biopax-level3.owl#Pathway,242
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,1104
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,2174
http://www.biopax.org/release/biopax-level3.owl#Rna,51
http://www.biopax.org/release/biopax-level3.owl#Interaction,6706
http://www.biopax.org/release/biopax-level3.owl#Protein,2652
http://www.biopax.org/release/biopax-level3.owl#Dna,0


18:34:54 INFO  Fuseki          :: [2] GET http://localhost:3030/humancyc_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of KEGG from PathwayCommons

In [11]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.kegg.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/kegg_pc']
process = subprocess.Popen(command)
time.sleep(60)
# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_kegg_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/kegg_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:35:56 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.kegg.BIOPAX.owl
18:35:57 WARN  riot            :: [line: 96269, col: 75] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:35:59 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:35:59 INFO  Server          :: Running in read-only mode for /kegg_pc
18:35:59 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:35:59 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:35:59 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:35:59 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:35:59 INFO  Server          :: 

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,1782
http://www.biopax.org/release/biopax-level3.owl#Pathway,122
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,0
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,1664
http://www.biopax.org/release/biopax-level3.owl#Rna,0
http://www.biopax.org/release/biopax-level3.owl#Interaction,3568
http://www.biopax.org/release/biopax-level3.owl#Protein,1872
http://www.biopax.org/release/biopax-level3.owl#Dna,0


18:36:57 INFO  Fuseki          :: [2] GET http://localhost:3030/kegg_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome

#### Query on BioPAX export of PID from PathwayCommons

In [12]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pid.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pid_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pid_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pid_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:37:59 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pid.BIOPAX.owl
18:38:00 WARN  riot            :: [line: 77671, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:38:04 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:38:04 INFO  Server          :: Running in read-only mode for /pid_pc
18:38:04 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:38:04 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:38:04 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:38:04 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:38:04 INFO  Server          :: Da

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,3065
http://www.biopax.org/release/biopax-level3.owl#Pathway,745
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,0
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,188
http://www.biopax.org/release/biopax-level3.owl#Rna,22
http://www.biopax.org/release/biopax-level3.owl#Interaction,16017
http://www.biopax.org/release/biopax-level3.owl#Protein,6584
http://www.biopax.org/release/biopax-level3.owl#Dna,0


18:39:01 INFO  Fuseki          :: [2] GET http://localhost:3030/pid_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.

#### Query on BioPAX export of INOH from PathwayCommons

In [13]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.inoh.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/inoh_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_inoh_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/inoh_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:40:04 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.inoh.BIOPAX.owl
18:40:05 WARN  riot            :: [line: 100079, col: 94] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:40:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:40:12 INFO  Server          :: Running in read-only mode for /inoh_pc
18:40:12 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:40:12 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:40:12 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:40:12 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:40:12 INFO  Server          ::

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,1586
http://www.biopax.org/release/biopax-level3.owl#Pathway,774
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,3427
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,4273
http://www.biopax.org/release/biopax-level3.owl#Rna,176
http://www.biopax.org/release/biopax-level3.owl#Interaction,5625
http://www.biopax.org/release/biopax-level3.owl#Protein,12804
http://www.biopax.org/release/biopax-level3.owl#Dna,588


18:41:07 INFO  Fuseki          :: [2] GET http://localhost:3030/inoh_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome

#### Query on BioPAX export of NetPath from PathwayCommons

In [14]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.netpath.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/netpath_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_netpath_pc)
sparql.setQuery(prefixes+query)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/netpath_pc_counts_bp_classes.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

18:42:09 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.netpath.BIOPAX.owl
18:42:11 WARN  riot            :: [line: 83771, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
18:42:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
18:42:12 INFO  Server          :: Running in read-only mode for /netpath_pc
18:42:12 INFO  Server          :: Apache Jena Fuseki 4.9.0
18:42:12 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
18:42:12 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run
18:42:12 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure3/Scripts/run/shiro.ini
18:42:12 INFO  Server       

class_of_interest,nb_instances
http://www.biopax.org/release/biopax-level3.owl#BiochemicalReaction,2014
http://www.biopax.org/release/biopax-level3.owl#Pathway,27
http://www.biopax.org/release/biopax-level3.owl#PathwayStep,0
http://www.biopax.org/release/biopax-level3.owl#SmallMolecule,0
http://www.biopax.org/release/biopax-level3.owl#Rna,0
http://www.biopax.org/release/biopax-level3.owl#Interaction,7214
http://www.biopax.org/release/biopax-level3.owl#Protein,4861
http://www.biopax.org/release/biopax-level3.owl#Dna,0


18:43:11 INFO  Fuseki          :: [2] GET http://localhost:3030/netpath_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.react